# 14 - Pass, containers and for

This notebook is a serie of exercises about the concept presented in [14 Pass, containers, for] and made by [Theo van Walsum](mailto:t.vanwalsum@erasmusmc.nl).

- Those exercises are not mandatory but it is strongly advised to do them as programming is skill learnt by doing
- Exercise have an associated difficulty level: 1 means that only an understanding of the course is sufficient to complete the exercise, 2 means that some research is needed to complete the exercise

## Exercise 1: Calculating term frequencies in the document
In information retrieval, [tf–idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (short for term frequency–inverse document frequency) is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. Another common problem is that frequently occuring words (e.g. articles, prepositions, and auxiliary verbs) can skew the statistics. They are commonly assembled into a stoplist which is then used to filter the document.

In the first part of the exercise we will focus on calculating the term frequencies for words from a single document. Term frequency (TF) is in its simplest form calculated by counting all occurences of a word in a document and then dividing it by a total number of words in the document. 

Create a function that takes a document filename as an argument and outputs all words from the document together with their term frequencies. Word case should be ignored, e.g. `Word` and `word` should be counted together. The function should return a single data structure. All words from a stoplist should not be counted in the calculations. You can create your own stoplist of at least 20 words or use one of publicly available stoplists. We provide you with a test file (`emma-tf.txt`)  where all punctuation marks have been removed and all words are separated by a space character. Pay close attention to your choice of data structures. Find the most frequent word in the text. Should it have been included in the stoplist you have used?  

In [1]:
# Solution 1.1
from pathlib import Path

def most_frequent_word(file_path, use_stop_list):
    """
    Sort all words in a text-file by frequency.

    arg1 = path to .txt file
    arg2 = whether to exclude words that are in the stop_list, or not
    """
    data_dict = {}
    stop_list = ['the', 'a', 'an', 'is', 'was', 'were', 'be', 'had', 'have',
                 'i', 'you', 'we', 'he', 'she', 'his', 'her', 'hers', 'this', 'that', 'there', 'they', 
                 'at', 'and', 'in', 'to', 'of', 'by', 'from', 'for', 'it', 'at', 'with', 
                 'but', 'now', 'not', 'so', 'as', 'very']

    with open(file_path) as file:
        data = file.read().lower()       # lees file als lowercase letters
        data_list = data.split(' ')      # split str op spaties --> maakt er een list van

        # ga voor elk woord tellen hoe vaak het voorkomt
        for word in data_list:
            try: 
                data_dict[word] += 1     # als de key nog niet bestaat, geeft dit een KeyError
            except KeyError:
                if use_stop_list:
                    # excludeer woorden in stop_list
                    if word not in stop_list:
                        data_dict[word] = 1  # 1e keer dat het geteld wordt
                else:
                    # we hoeven niks te excluderen, tel gewoon alles
                    data_dict[word] = 1  # 1e keer dat het geteld wordt
        
        # sorteer de dictionary op 'vaakst geteld'
        sorted_list = sorted(data_dict.items(), key=lambda item: item[1], reverse=True)
        print(f"Most frequent word is '{sorted_list[0][0]}' with {sorted_list[0][1]} times \n")

        return sorted_list[:100]  # print alleen de eerste 100 omdat je anders kunt scrollen tot je een ons weegt


file_path = Path('data_14_pass_containers_for') / 'emma-tf.txt'  # definieer pad naar .txt file
print(most_frequent_word(file_path, True))


Most frequent word is 'mr.' with 1153 times 

[('mr.', 1153), ('emma', 862), ('all', 846), ('could', 837), ('would', 820), ('him', 769), ('been', 760), ('no', 742), ('my', 732), ('mrs.', 700), ('on', 691), ('any', 655), ('do', 635), ('miss', 602), ('me', 583), ('must', 571), ('will', 559), ('which', 556), ('what', 536), ('harriet', 505), ('or', 494), ('such', 489), ('much', 486), ('if', 485), ('said', 484), ('more', 468), ('are', 455), ('one', 453), ('weston', 439), ('every', 435), ('them', 433), ('am', 425), ('than', 416), ('thing', 398), ('knightley', 389), ('elton', 387), ('think', 384), ('well', 381), ('how', 371), ('should', 371), ('your', 367), ('when', 364), ('little', 360), ('being', 358), ('never', 358), ('did', 353), ('only', 341), ('know', 337), ('might', 326), ('good', 313), ('woodhouse', 312), ('say', 311), ('their', 306), ('own', 303), ('jane', 300), ('who', 295), ('can', 283), ('quite', 282), ('herself', 277), ('time', 277), ('some', 265), ('great', 265), ('too', 254), (

## Exercise 2: Markov analysis (level of difficulty: 2)

This exercise is inspired by / taken from the 'Think Python' (Allen B. Downey). 

In a Markov process, transition to the next state is only determined by the current state. Such Markov processes can be used to model e.g. text. In this exercises, you will implement such a text processing modeling system, and use it to generate text.

In text processing, the 'current state' can be represented by the previous N words, and the predicted new state is the next word. We can build such a model by reading a text, and then for every set of N words, store each next word (or better: all possible next words, including their frequency). 

The first exercise is to create a function that builds such a model when reading a text file. The main task is to decide on the appropriate datastructures for building this model. In building the model, you must also take into account that the same model will be used to generate new text from this model.

As an input file, the [emma.txt](https://github.com/AllenDowney/ThinkPython2/blob/master/code/emma.txt) has been prepared. All dashes, underscores and quotation marks have been stripped. What remains is text, with upper and lower case characters, and the following punctuation marks: ' , . : ; ! ? . For this exercise, you may consider these punctuation marks as words. For your convenience, the input text has been prepared such that these punctuation marks are separated by spaces before and after, such that all input words are space (space / newline / tab) separated. You do not need to change the case of the characters.

In addition, there is a simpler text file, called test.txt in the data_14 folder. You may use that to e.g. print the result of the model building.



In [2]:
# Solution 2.1
from pathlib import Path

def generate_model(file_path):
    giant_list = {}

    with open(Path('data_14_pass_containers_for') / file_path) as file:
        text = file.read()
        text_list = text.replace('\n', ' ').split(' ')

        for index, word in enumerate(text_list):
            if index < len(text_list) - 1:
                try:
                    giant_list[word].append(text_list[index+1])
                except:
                    giant_list[word] = []
                    giant_list[word].append(text_list[index+1])
    return giant_list


file_path = 'test.txt'
print(generate_model(file_path))


{'Dit': ['is'], 'is': ['een'], 'een': ['simpel'], 'simpel': ['test', 'test'], 'test': ['bestand', 'bestand'], 'bestand': ['voor', 'kan'], 'voor': ['het'], 'het': ['opbouwen', 'Markov', 'Markov', 'Markov', 'Markov'], 'opbouwen': ['van'], 'van': ['het'], 'Markov': ['model', 'model', 'model', 'model'], 'model': ['.', 'inderdaad', 'moet', 'geen'], '.': ['Met', 'Want'], 'Met': ['zo'], 'zo': ["'"], "'": ['n'], 'n': ['simpel'], 'kan': ['je', 'je'], 'je': ['kijken', 'met'], 'kijken': ['of'], 'of': ['het'], 'inderdaad': ['goed'], 'goed': ['gemaakt'], 'gemaakt': ['wordt'], 'wordt': ['.'], 'Want': ['het'], 'moet': ['natuurlijk'], 'natuurlijk': ['wel'], 'wel': ['kloppen'], 'kloppen': [','], ',': ['anders'], 'anders': ['kan'], 'met': ['het'], 'geen': ['zinnen'], 'zinnen': ['vormen'], 'vormen': ['.']}


Write code to create random sentences. Think of a good way to initialize the first state in a nice and random manner.

In [5]:
# Solution 2.2
import random
from time import sleep

def text_processing(file_path, num_words):
    # definieer mogelijke woorden om de output mee te beginnen...
    start_list = ['This', 'He', 'She', 'They', 'You', 'I', 'So', 'But', 'However', 'It', 'Emma', 'Mr.', 'Mrs.' 
                  if file_path == 'emma.txt' else 'Dit', 'Met', 'Want']
    # ... en kies random eentje
    start_word = random.choice(start_list) 

    # 'train' het model op de .txt file
    model = generate_model(file_path)

    # print vast het 1e woord
    output = start_word
    print(output, end=' ')

    for i in range(num_words):
        # uit de array van mogelijke woorden na het vorige woord, dus output.split(' ')[-1] ...
        # ... kies een random woord
        new_word = random.choice(model[output.split(' ')[-1]])

        # voeg toe aan huidige output, en print op dezelfde regel
        output = output + " " + new_word
        print(new_word, end=' ')

        # random delay, net als ChatGPT, omdat het kan
        sleep(random.choice([0, 0.02, 0.05, 0.07]))  # in seconden


file_path = 'emma.txt'  # input file
num_words = 200         # hoeveel woorden het moet genereren
text_processing(file_path, num_words)


He thought so miserable state of from my uncle. Without knowing that I can it is, that it had been unnoticed, because she has been with a word might be kept it to be enduring by Miss Bates loved by their friendship for it not. But her taste of affection to value of hours; he had been a want to her life would be calling soon. Every body who had been so well class of Miss Taylor too; for though her convictions, with him. The good manners. Sighs and break up to expect that you were most excellent, and Miss Bates in a little more of her at the rest; and tried to prevent the two ladies of the distinguished honour of the wonder. Mr. John had often, when a distinct parties;--with so very delightful woman!--A style was equally ill-bestowed. It is nothing ungallant, nothing more useful now.”  “The joke,” he must not to a caution.--I wish to spend the first attempt on good wife. Remember.”    Redistribution is not play whenever it would be _broke_ to me keep the hall. Harriet felt immediately af

# Exercise 2: Markov model for words (level of difficulty: 2)

The same model building approach can be applied to modeling words instead of sentences. Do the same for words. In reading words, only include words of at least key_len + 1 char, and convert all words to lower case. 

In [ ]:
# Solution 3.1

In [ ]:
# Solution 3.2


Given the models build, you may want to get more information. E.g. which state has the highest number of possible next words. And how many different words are in that list. And how many states do have exactly one (deterministic) subsequent word. And what is the histogram of the number of words per state (i.e. visualize in a graph the number of states with exactly one word in the list, and with two words in the list, etc). And how do these histograms vary, e.g. as a function of key_len (what do you expect), of text (or author), or of language. It is left as a further exercise (without solution) to program solutions that may answer such questions. 